# 动量信号模型演示

本演示展示了如何使用动量信号模型预测市场趋势变化，并通过SHAP监控因子表现，最终通过Optuna动态调节超参数。

## 项目结构
- **data**: 数据生成模块
- **factors**: 动量因子计算模块
- **model**: 市场趋势预测模型
- **optimization**: 超参数优化模块
- **explanation**: 模型解释模块
- **evaluation**: 模型评估模块
- **utils**: 工具模块


In [ ]:
# 导入必要的库
import pandas as pd
import numpy as np

# 导入项目模块
from data.data_generator import generate_simulated_data
from factors.momentum_factors import calculate_momentum_factors
from model.predictor import MarketTrendPredictor
from optimization.hyperparameter_tuning import optimize_hyperparameters
from explanation.model_explanation import create_shap_explainer, calculate_shap_values, analyze_factor_importance
from evaluation.metrics import calculate_performance_metrics

print('模块导入成功')

## 1. 数据生成

首先生成模拟的股票数据用于演示。在实际应用中，可以替换为真实的历史数据。

In [ ]:
# 生成模拟数据
raw_data = generate_simulated_data(n_stocks=20, n_days=500)

print(f'生成了 {len(raw_data)} 条数据记录')
print(f'数据时间范围: {raw_data["date"].min()} 到 {raw_data["date"].max()}')
print(f'股票数量: {raw_data["stock_code"].nunique()}')

# 显示部分数据
raw_data.head(10)

## 2. 动量因子计算

计算短、中、长期动量因子（21/63/252交易日窗口）

In [ ]:
# 计算动量因子
factor_data = calculate_momentum_factors(raw_data)

print(f'因子数据记录数: {len(factor_data)}')
print('因子列:', list(factor_data.columns))

# 显示部分因子值
factor_data[['stock_code', 'date', 'close_price', 'short_momentum', 'medium_momentum', 'long_momentum']].head(10)

## 3. 模型训练与预测

使用随机森林回归模型训练市场趋势预测器

In [ ]:
# 创建预测器
predictor = MarketTrendPredictor()

# 准备特征和目标变量
features = predictor.prepare_features(factor_data)
target = predictor.prepare_target(factor_data)

print(f'特征数据形状: {features.shape}')
print(f'目标变量形状: {target.shape}')

# 训练模型
predictor.train(features, target)
print('模型训练完成')

# 进行预测
predictions = predictor.predict(features)
print(f'预测结果形状: {predictions.shape}')

## 4. 模型评估

评估模型性能，包括MSE、RMSE和信息比率(IR)

In [ ]:
# 评估模型
evaluation_metrics = predictor.evaluate(features, target)

print('模型评估指标:')
for metric, value in evaluation_metrics.items():
    print(f'  {metric}: {value:.6f}')

# 计算预测收益率的综合性能指标
performance_metrics = calculate_performance_metrics(pd.Series(predictions))

print('
预测收益率综合性能指标:')
for metric, value in performance_metrics.items():
    print(f'  {metric}: {value:.6f}')

## 5. 超参数优化

使用Optuna进行超参数优化，以提高模型性能

In [ ]:
# 执行超参数优化
# 注意：在实际应用中，可以增加试验次数以获得更好的优化结果
optimization_result = optimize_hyperparameters(features, target, n_trials=10)

print('超参数优化结果:')
print(f'最优参数: {optimization_result["best_params"]}')
print(f'最优值 (RMSE): {optimization_result["best_value"]:.6f}')

# 使用最优参数更新模型
predictor.update_model_parameters(optimization_result['best_params'])
print('模型参数已更新')

## 6. 模型解释

使用SHAP分析因子重要性，监控市场趋势变化

In [ ]:
# 创建SHAP解释器
explainer = create_shap_explainer(predictor, features)

# 计算SHAP值
shap_values = calculate_shap_values(explainer, features)

print(f'SHAP值形状: {shap_values.shape}')

# 分析因子重要性
importance_df = analyze_factor_importance(shap_values, features.columns.tolist())

print('因子重要性排序:')
importance_df

## 7. 结论

本演示展示了动量信号模型的完整工作流程：
1. 数据生成/加载
2. 动量因子计算
3. 模型训练与预测
4. 模型评估
5. 超参数优化
6. 模型解释

通过以上步骤，我们可以：
- 预测市场趋势变化
- 监控短、中、长期动量因子的表现
- 通过超参数优化提高模型性能
- 使用SHAP解释模型决策，检测市场趋势变化

模型价值通过以下指标体现：
- 信息比率(IR)
- RMSE
- 因子重要性排序
- SHAP值分析